## A. Introduction

### A.1. Disscusion of the Background

I have decided to use Atlanta in my project. Since I live in nearby Atlanta, and Atlanta is one of the largest, and fast growing metropolis in Southeastern United States, I thought it would be interesting to analyze Parking lot data and come up with possible Parking Price solutions by clustering parking lots by distance from city center.

### A.2 Data Description  

Based on definition of our goal, we should focus on these important factors:
* Number of existing neighborhoods or Borough, and number of existing business by business type in neighborhood.

Following data sources will be used:
* Forsquare API
* Geolocator



## Importing Libraries

In [1]:
!pip install lxml
!pip install geocoder
!pip install shapely
!pip install pyproj


In [2]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import lxml
import requests
import geocoder
import shapely.geometry
import pyproj
import math
from pandas.io.json import json_normalize

In [4]:
CLIENT_ID = 'CZGYUPXL0PIGYAXUJCH4TNM21YE4ZPLZCOEVJYKCUAP3OSRT' # your Foursquare ID
CLIENT_SECRET = 'JRIYIMKCPHM3ZONUR1IN4XJYYI51ATAJV1SPUKEM333GZANL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CZGYUPXL0PIGYAXUJCH4TNM21YE4ZPLZCOEVJYKCUAP3OSRT
CLIENT_SECRET:JRIYIMKCPHM3ZONUR1IN4XJYYI51ATAJV1SPUKEM333GZANL


In [5]:
address = 'Atlanta, GA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

33.7490987 -84.3901849


In [6]:
# create map of Atlanta using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=11)

map

In [7]:
LIMIT=100
radius =4000
search_query='parking'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    latitude, 
    longitude, 
    VERSION,
    search_query,
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CZGYUPXL0PIGYAXUJCH4TNM21YE4ZPLZCOEVJYKCUAP3OSRT&client_secret=JRIYIMKCPHM3ZONUR1IN4XJYYI51ATAJV1SPUKEM333GZANL&ll=33.7490987,-84.3901849&v=20180605&query=parking&radius=4000&limit=100'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5de6a881be61c9001b450205'},
 'response': {'venues': [{'id': '4e2ce96c6284b96d7b8e3fa5',
    'name': 'Grady Memorial Hospital Parking Deck',
    'location': {'lat': 33.75274029400171,
     'lng': -84.38210333937693,
     'labeledLatLngs': [{'label': 'display',
       'lat': 33.75274029400171,
       'lng': -84.38210333937693}],
     'distance': 850,
     'cc': 'US',
     'city': 'Atlanta',
     'state': 'GA',
     'country': 'United States',
     'formattedAddress': ['Atlanta, GA', 'United States']},
    'categories': [{'id': '4c38df4de52ce0d596b336e1',
      'name': 'Parking',
      'pluralName': 'Parking',
      'shortName': 'Parking',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/parking_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1575397505',
    'hasPerk': False},
   {'id': '4b847673f964a520f73631e3',
    'name': 'CNN Center Upper Deck Parking Lot',
    'location': {'address': 'Centennia

In [9]:
Parking = results['response']['venues']
Parking = json_normalize(Parking) # flatten JSON
Parking.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode,location.crossStreet,location.neighborhood
0,4e2ce96c6284b96d7b8e3fa5,Grady Memorial Hospital Parking Deck,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",v-1575397505,False,33.752740,-84.382103,"[{'label': 'display', 'lat': 33.75274029400171...",850,US,Atlanta,GA,United States,"[Atlanta, GA, United States]",NaN,NaN,NaN,NaN
1,4b847673f964a520f73631e3,CNN Center Upper Deck Parking Lot,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",v-1575397505,False,33.756496,-84.395311,"[{'label': 'display', 'lat': 33.7564956003286,...",950,US,Atlanta,GA,United States,"[Centennial Olympic Park Dr NW, Atlanta, GA 30...",Centennial Olympic Park Dr NW,30303,NaN,NaN
2,4c09bf36a1b32d7fd64f98f0,Georgia State Capitol Parking Deck,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",v-1575397505,False,33.748590,-84.387558,"[{'label': 'display', 'lat': 33.74859022429047...",249,US,Atlanta,GA,United States,[Martin Luther King Jr Dr SE (at Capitol Ave S...,Martin Luther King Jr Dr SE,30334,at Capitol Ave SE,NaN
3,4c7bdd6a794e224bfe836c28,98 Cone Street Parking,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",v-1575397505,False,33.757333,-84.390997,"[{'label': 'display', 'lat': 33.75733334734233...",919,US,Atlanta,GA,United States,"[Cone Street, Atlanta, GA, United States]",Cone Street,NaN,NaN,NaN
4,4adf2617f964a520dc7721e3,Lanier Parking Solutions,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",v-1575397505,False,33.760721,-84.386394,"[{'label': 'display', 'lat': 33.76072144847832...",1340,US,Atlanta,GA,United States,"[233 Peachtree St, 2600 Harris Tower, Atlanta,...","233 Peachtree St, 2600 Harris Tower",30303,NaN,NaN


In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng','location.distance']
Parking =Parking.loc[:, filtered_columns]

# filter the category for each row
Parking['categories'] = Parking.apply(get_category_type, axis=1)

# clean columns
Parking.columns = [col.split(".")[-1] for col in Parking.columns]

Parking

,name,categories,lat,lng,distance
0,Grady Memorial Hospital Parking Deck,Parking,33.752740,-84.382103,850
1,CNN Center Upper Deck Parking Lot,Parking,33.756496,-84.395311,950
2,Georgia State Capitol Parking Deck,Parking,33.748590,-84.387558,249
3,98 Cone Street Parking,Parking,33.757333,-84.390997,919
4,Lanier Parking Solutions,Parking,33.760721,-84.386394,1340
5,GSU Lofts Parking,Parking,33.754055,-84.383254,846
6,Red Deck Parking,Parking,33.756878,-84.398914,1184
7,GSU Parking - U Lot,Parking,33.752892,-84.387030,513
8,Peachtree Center Parking Garage - AmeriPark,Parking,33.759849,-84.385618,1269
9,Central Parking,Parking,33.753841,-84.390247,527


In [12]:
#Change Categories to parking 
Parking.loc[Parking['categories'] != "Parking"]                               

,name,categories,lat,lng,distance
16,Auburn Ave Public Parking,General Travel,33.755932,-84.386397,837
24,Government Center Parking Deck,General Travel,33.747751,-84.391050,170
25,Lanier Parking,Office,33.759534,-84.388448,1172
32,Parking Meter #101-023-20,Road,33.750122,-84.391541,169
35,Parking Garage,Travel Lounge,33.747237,-84.391304,231
38,CNN Center Parking Deck Bridge,Bridge,33.756833,-84.395260,980
41,Atlanta Medical Center - Towne Park Parking Of...,Medical Center,33.763255,-84.372275,2287
44,Freedom Parkway Parking Lot,Road,33.760108,-84.375112,1856
47,Atlanta Public Schools Parking Lot,General Travel,33.748184,-84.392524,239
48,MARTA - Five Points Station,Metro Station,33.753919,-84.391550,551


In [13]:
# Copy Categories to parking cases 

idx_to_change = Parking.loc[Parking['categories'] != "Parking"].index
for i in idx_to_change:
    Parking.iloc[i, 1] = Parking.iloc[i-1, 1] 

In [14]:
Parking.tail()

,name,categories,lat,lng,distance
45,AAA Parking Deck,Parking,33.752975,-84.388489,459
46,191 Parking Deck,Parking,33.758275,-84.386506,1076
47,Atlanta Public Schools Parking Lot,Parking,33.748184,-84.392524,239
48,MARTA - Five Points Station,Parking,33.753919,-84.391550,551
49,Century Skyline Parking Garage,Parking,33.764729,-84.382548,1878


In [15]:
# create Parking_map of Atlanta using latitude and longitude values
Parking_map = folium.Map(location=[latitude, longitude], zoom_start=11)


# add a red circle marker to represent the Basílica de la Sagrada Familia
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Atlnata',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(Parking_map)


# add markers to map
for lat, lng, label in zip(Parking['lat'], Parking['lng'], Parking['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Parking_map)  
    
Parking_map

In [16]:
Parking.head()

,name,categories,lat,lng,distance
0,Grady Memorial Hospital Parking Deck,Parking,33.752740,-84.382103,850
1,CNN Center Upper Deck Parking Lot,Parking,33.756496,-84.395311,950
2,Georgia State Capitol Parking Deck,Parking,33.748590,-84.387558,249
3,98 Cone Street Parking,Parking,33.757333,-84.390997,919
4,Lanier Parking Solutions,Parking,33.760721,-84.386394,1340


In [17]:
# set number of clusters
kclusters = 3
Parking_grouped_clustering = Parking.drop(['name','categories'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Parking_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 1, 1, 2,
       2, 2, 1, 2, 2, 0, 0, 2, 0, 1, 1, 2, 0, 1, 1, 1, 2, 0, 1, 0, 0, 1,
       0, 1, 2, 1, 1, 0], dtype=int32)

In [18]:
# add clustering labels
Parking.insert(0, 'Cluster Labels', kmeans.labels_)


In [19]:
Parking.head()

,Cluster Labels,name,categories,lat,lng,distance
0,2,Grady Memorial Hospital Parking Deck,Parking,33.752740,-84.382103,850
1,2,CNN Center Upper Deck Parking Lot,Parking,33.756496,-84.395311,950
2,1,Georgia State Capitol Parking Deck,Parking,33.748590,-84.387558,249
3,2,98 Cone Street Parking,Parking,33.757333,-84.390997,919
4,2,Lanier Parking Solutions,Parking,33.760721,-84.386394,1340


### B. Disscusion of the Result

In [22]:
# create Parking_map of Atlanta using latitude and longitude values
Parking_map = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# add a red circle marker to represent the Basílica de la Sagrada Familia
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='yellow',
    popup='Atlnata',
    fill = True,
    fill_color = 'yellow',
    fill_opacity = 0.6
).add_to(Parking_map)


# add markers to map
for lat, lng, label,cluster in zip(Parking['lat'], Parking['lng'], Parking['name'], Parking['Cluster Labels']):
    label = folium.Popup(str(label)+ ': Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(Parking_map)  
    
Parking_map

## B.1 Cluster 0

In [34]:
Parking.loc[Parking['Cluster Labels'] == 0]

,Cluster Labels,name,categories,lat,lng,distance
11,0,WoCC Parking Garage,Parking,33.764308,-84.393871,1727
13,0,Georgia Aquarium Parking Deck,Parking,33.764292,-84.395779,1768
27,0,GT Parking - Peters Parking Deck (Zone E44),Parking,33.775439,-84.393987,2953
28,0,GT Parking - Area 1,Parking,33.771277,-84.394660,2503
30,0,Spelman College Parking Garage,Parking,33.745366,-84.412899,2143
34,0,GT Parking Deck - Zone W02,Parking,33.774025,-84.400017,2920
39,0,GT Parking - Area 4,Parking,33.777895,-84.398680,3300
41,0,Atlanta Medical Center - Towne Park Parking Of...,Parking,33.763255,-84.372275,2287
42,0,Central Parking,Parking,33.734105,-84.393712,1700
44,0,Freedom Parkway Parking Lot,Parking,33.760108,-84.375112,1856


## B.2 Cluster 1

In [35]:
Parking.loc[Parking['Cluster Labels'] == 1]

,Cluster Labels,name,categories,lat,lng,distance
2,1,Georgia State Capitol Parking Deck,Parking,33.748590,-84.387558,249
7,1,GSU Parking - U Lot,Parking,33.752892,-84.387030,513
9,1,Central Parking,Parking,33.753841,-84.390247,527
19,1,Pete Hackney Parking Garage,Parking,33.750613,-84.384175,581
20,1,Underground Atlanta Parking Deck,Parking,33.751291,-84.390255,244
24,1,Government Center Parking Deck,Parking,33.747751,-84.391050,170
31,1,Butler Parking Deck,Parking,33.749119,-84.384741,503
32,1,Parking Meter #101-023-20,Parking,33.750122,-84.391541,169
35,1,Parking Garage,Parking,33.747237,-84.391304,231
36,1,Parking Lot,Parking,33.750398,-84.388020,247


## B.3 Cluster 2

In [36]:
Parking.loc[Parking['Cluster Labels'] == 2]

,Cluster Labels,name,categories,lat,lng,distance
0,2,Grady Memorial Hospital Parking Deck,Parking,33.752740,-84.382103,850
1,2,CNN Center Upper Deck Parking Lot,Parking,33.756496,-84.395311,950
3,2,98 Cone Street Parking,Parking,33.757333,-84.390997,919
4,2,Lanier Parking Solutions,Parking,33.760721,-84.386394,1340
5,2,GSU Lofts Parking,Parking,33.754055,-84.383254,846
6,2,Red Deck Parking,Parking,33.756878,-84.398914,1184
8,2,Peachtree Center Parking Garage - AmeriPark,Parking,33.759849,-84.385618,1269
10,2,Georgia-Pacific Parking Garage,Parking,33.757510,-84.385482,1032
12,2,Parking Garage,Parking,33.756556,-84.387758,860
14,2,Middle Earth Parking Deck,Parking,33.755611,-84.395290,865


## C. Result

By clustering them into 3, we were able to analyze data. The distance from city center determines the price range of the parking lots. We can easily use this outcome to create recommendation system that is sorted by distance. 